In [1]:
from table_schema import generate_schema_prompt
from mysql_setup import perform_query_on_mysql_databases

## Check Schema


In [2]:
result = generate_schema_prompt(
    sql_dialect="MySQL", db_name="superhero", table_name="colour"
)

In [3]:
print(result)

CREATE TABLE `colour` (
  `id` int NOT NULL,
  `colour` text,
  PRIMARY KEY (`id`)
)

Example data:
 id    colour
  1 No Colour
  2     Amber
  3    Auburn


## Reproduce the error


In [5]:
error_query = """INSERT INTO colour(id, colour) VALUES(
    (999, 'Red999'),
    (111, 'Blue111')
);"""

In [6]:
perform_query_on_mysql_databases(error_query,"superhero")

OperationalError: (1241, 'Operand should contain 1 column(s)')

## Reference Solution


In [7]:
sol_query = """INSERT INTO colour(id,colour) VALUES
    (999, 'Red999'),
    (111, 'Blue111')
;"""

In [10]:
perform_query_on_mysql_databases(sol_query,"superhero")

[]

## Test Cases


In [12]:
def test_colour_insertion():
    # Test case 1: Check if the new records exist
    query = "SELECT * FROM colour WHERE id IN (999, 111)"
    results = perform_query_on_mysql_databases(query,"superhero")
    assert (
        len(results) == 2
    ), f"""Expected 2 new records, but found {
        len(results)}"""

    # Test case 2: Verify the content of the first inserted record
    query = "SELECT * FROM colour WHERE id = 999"
    result = perform_query_on_mysql_databases(query,"superhero")
    assert (
        result[0][0] == 999
    ), f"""Expected id 999, but got {
        result[0][0]}"""
    assert (
        result[0][1] == "Red999"
    ), f"""Expected colour 'Red999', but got {
        result[0][1]}"""

    # Test case 3: Verify the content of the second inserted record
    query = "SELECT * FROM colour WHERE id = 111"
    result = perform_query_on_mysql_databases(query,"superhero")
    assert (
        result[0][0] == 111
    ), f"""Expected id 111, but got {
        result[0][0]}"""
    assert (
        result[0][1] == "Blue111"
    ), f"Expected colour 'Blue111', but got {result[0][1]}"

    # Test case 4: Check the total count of records in the table
    query = "SELECT COUNT(*) as total FROM colour"
    result = perform_query_on_mysql_databases(query,"superhero")
    assert (
        result[0][0] > 2
    ), f"Expected more than 2 total records, but found {result[0][0]}"

    # Test case 5: Attempt to insert a duplicate id (should fail)
    query = "INSERT INTO colour(id, colour) VALUES (999, 'DuplicateRed')"
    try:
        perform_query_on_mysql_databases(query,"superhero")
        assert False, "Inserting duplicate id should have raised an error"
    except Exception as err:
        assert "Duplicate entry" in str(
            err
        ), f"Expected 'Duplicate entry' error, but got: {err}"

    print("All test cases passed successfully!")


# Usage:
test_colour_insertion()

All test cases passed successfully!


## Clean up


In [13]:
cleanup_query = "DELETE FROM colour WHERE id IN (999, 111)"
perform_query_on_mysql_databases(cleanup_query,"superhero")

# Verify cleanup
verify_query = "SELECT COUNT(*) as count FROM colour WHERE id IN (999, 111)"
result = perform_query_on_mysql_databases(verify_query,"superhero")
remaining_count = result[0][0]
if remaining_count == 0:
    print("Cleanup successful: All test records removed.")
else:
    print(f"Cleanup incomplete: {remaining_count} test records remain.")

Cleanup successful: All test records removed.
